# [Tokenizers库](https://huggingface.co/learn/nlp-course/zh-CN/chapter6)
当我们需要微调模型时，我们需要使用与模型预训练相同的`tokenizer`

## 基于已有的 tokenizer 训练新的 tokenizer
大多数`Transformer 模型`使用`子词分词算法`。为了找到语料库中的常见子词，`tokenizer`需要深入统计语料库中的所有文本——这个过程我们称之为`训练（training）`。具体的训练规则取决于使用的`tokenizer`类型。
> 训练 tokenizer 与训练模型不同！模型训练使用随机梯度下降使每个 batch 的 loss 小一点。它本质上是随机的（这意味着在即使两次训练的参数和算法完全相同，你也必须设置一些随机数种子才能获得相同的结果）。训练 tokenizer 是一个统计过程，它试图确定哪些子词最适合为给定的语料库选择，确定的过程取决于分词算法。它是确定性的，这意味着在相同的语料库上使用相同的算法进行训练时，得到的结果总是相同的。

In [ ]:
# step1 准备语料库
from datasets import load_dataset

# # 下载不下来数据的时候可以增加魔法
# import os
# os.environ["http_proxy"] = "http://127.0.0.1:port"
# os.environ["https_proxy"] = "http://127.0.0.1:port"

# # 加载数据集
raw_datasets = load_dataset("code_search_net", "python", trust_remote_code=True)
print(raw_datasets["train"])

In [ ]:
# 我们仅使用 whole_func_string 列来训练我们的 tokenizer 
# 我们可以通过索引来查看其中一个函数的示例
print(raw_datasets["train"][123456]["whole_func_string"])


In [ ]:
# 创建迭代器
# 将数据集转换为一个文本列表的iterator（迭代器），这样就可以以batch的方式进行训练了
# 使用迭代器可以不把所有内容都加载到内存中
# 通过 python 的 generator 来实现
## version 1 基于列表的 generator
def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["whole_func_string"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )


training_corpus = get_training_corpus()
print(training_corpus)

## version 2 基于 yield 的 generator
def get_training_corpus_v2():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]
    
training_corpus_v2 = get_training_corpus_v2()
print(training_corpus_v2)

In [ ]:
# step2 训练一个新的 tokenizer
from transformers import AutoTokenizer
# 使用旧的 tokenizer，训练一个新的 tokenizer
# 新的 tokenizer 将与 GPT-2 完全相同，唯一的区别是词汇表，这将由我们的语料库通过训练来重新确定。
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
# 展示在旧的 tokenizer 下的 token
print(tokens)
# 使用code_search_net中的Python数据集，训练一个新的 tokenizer ，词汇表大小为52000
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)
tokens = tokenizer.tokenize(example)
print(tokens)
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

## Token-classification
`Token-classification`是自然语言处理（NLP）中的核心任务，旨在为文本中的每个 token（如单词、子词或字符）分配一个特定的标签。它的核心目标是通过对文本的细粒度分析，提取结构化信息或理解语言单元的语义角色，广泛应用于命名实体识别（NER）、分词、词性标注等场景。    


In [ ]:
# 使用pipeline，默认基于 dbmdz/bert-large-cased-finetuned-conll03-english 模型对句子进行 NER
from transformers import pipeline

token_classifier = pipeline("token-classification")
print(token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn."))
# 可以使用将同一实体的 token 组合在一起
## aggregation_strategy 通过设置，可以设置组合后实体计算的策略
token_classifier = pipeline("token-classification", aggregation_strategy="simple")
print(token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn."))

In [ ]:
# 自己构建pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
inputs = tokenizer(example, return_tensors="pt")
outputs = model(**inputs)
# 输入的token的维度
print(inputs["input_ids"].shape)
# 输出的维度
print(outputs.logits.shape)
# 打印索引到标签的映射
print(model.config.id2label)

import torch
# 使用 softmax 函数将这些 logits 转化为概率，并取 argmax 来得到预测
probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0].tolist()
predictions = outputs.logits.argmax(dim=-1)[0].tolist()
print(predictions)

# 格式化输出 token 的得分和标签
results = []
# 获取偏移映射
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets["offset_mapping"]

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O":
        start, end = offsets[idx]
        results.append(
            {
                "entity": label,
                "score": probabilities[idx][pred],
                "word": tokens[idx],
                "start": start,
                "end": end,
            }
        )

print(results)

# 格式化输出组合后 token 的得分和标签
import numpy as np

results = []
idx = 0
while idx < len(predictions):
    pred = predictions[idx]
    label = model.config.id2label[pred]
    if label != "O":
        # 删除 B- 或者 I-
        label = label[2:]
        start, _ = offsets[idx]

        # 获取所有标有 I 标签的token
        all_scores = []
        while (
            idx < len(predictions)
            and model.config.id2label[predictions[idx]] == f"I-{label}"
        ):
            all_scores.append(probabilities[idx][pred])
            _, end = offsets[idx]
            idx += 1

        # 分数是该分组实体中所有token分数的平均值
        score = np.mean(all_scores).item()
        word = example[start:end]
        results.append(
            {
                "entity_group": label,
                "score": score,
                "word": word,
                "start": start,
                "end": end,
            }
        )
    idx += 1

print(results)

## question-answering
question-answering pipeline 用于执行问答任务。具体来说，它用于从给定的文本中提取答案，基于一个预训练的模型。


In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
context = """
🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch, and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
long_context = """
🤗 Transformers: State of the Art NLP

🤗 Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction,
question answering, summarization, translation, text generation and more in over 100 languages.
Its aim is to make cutting-edge NLP easier to use for everyone.

🤗 Transformers provides APIs to quickly download and use those pretrained models on a given text, fine-tune them on your own datasets and
then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and
can be modified to enable quick research experiments.

Why should I use transformers?

1. Easy-to-use state-of-the-art models:
  - High performance on NLU and NLG tasks.
  - Low barrier to entry for educators and practitioners.
  - Few user-facing abstractions with just three classes to learn.
  - A unified API for using all our pretrained models.
  - Lower compute costs, smaller carbon footprint:

2. Researchers can share trained models instead of always retraining.
  - Practitioners can reduce compute time and production costs.
  - Dozens of architectures with over 10,000 pretrained models, some in more than 100 languages.

3. Choose the right framework for every part of a model's lifetime:
  - Train state-of-the-art models in 3 lines of code.
  - Move a single model between TF2.0/PyTorch frameworks at will.
  - Seamlessly pick the right framework for training, evaluation and production.

4. Easily customize a model or an example to your needs:
  - We provide examples for each architecture to reproduce the results published by its original authors.
  - Model internals are exposed as consistently as possible.
  - Model files can be used independently of the library for quick experiments.

🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""

question = "Which deep learning libraries back 🤗 Transformers?"
print(question_answerer(question=question, context=context))
# 长文本也能处理
print(question_answerer(question=question, context=long_context))

In [ ]:
# 自己构建pipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_checkpoint = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

# 输入格式是：[CLS] question [SEP] context [SEP] 
inputs = tokenizer(question, context, return_tensors="pt")
outputs = model(**inputs)
# 问答模型训练的目标是来预测答案开始的 token 的索引和答案结束的 token 的索引。
# 所以返回值是两个logits：一个对应于答案的开始 token 的 logits，另一个对应于答案的结束 token 的 logits。
start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

# 使用 softmax 函数将这些 logits 转化为概率，并取 argmax 来得到预测
"""
注意：
由于输入格式是 [CLS] question [SEP] context [SEP] ，所以我们需要屏蔽 question 的 tokens 以及 [SEP] token 。
不过，我们将保留 [CLS] ，因为某些模型使用它来表示答案不在上下文中。
"""
import torch

sequence_ids = inputs.sequence_ids()
# 屏蔽除 context 之外的所有内容
mask = [i != 1 for i in sequence_ids]
# 不屏蔽 [CLS] token
mask[0] = False
mask = torch.tensor(mask)[None]
print(mask)
# 将想要屏蔽的 logits 替换为一个大的负数
start_logits[mask] = -10000
end_logits[mask] = -10000
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)[0]
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)[0]
print(start_probabilities.shape, end_probabilities.shape)

# 在满足 start_index <= end_index 的前提下，计算每个可能的 start_index 和 end_index 的概率，
# 然后取概率最高的 (start_index, end_index) 元组。
# start_probabilities[start_index] × end_probabilities[end_index] 就是这个 (start_index, end_index) 元组的概率。
scores = start_probabilities[:, None] * end_probabilities[None, :]
print(scores.shape)
import numpy as np
# 获取上三角部分
scores = torch.triu(scores)

# argmax 将返回展平（flattened）后张量中的索引
max_index = scores.argmax().item()
start_index = max_index // scores.shape[1]
end_index = max_index % scores.shape[1]
print(start_index, end_index)
print(scores[start_index, end_index])

# 有了start_index 和 end_index，可以从 context 中提取答案
inputs_with_offsets = tokenizer(question, context, return_offsets_mapping=True)
offsets = inputs_with_offsets["offset_mapping"]

start_char, _ = offsets[start_index]
_, end_char = offsets[end_index]
answer = context[start_char:end_char]
# 格式化输出
result = {
    "answer": answer,
    "start": start_char,
    "end": end_char,
    "score": scores[start_index, end_index],
}
print(result)

In [ ]:
offsets[start_index]

In [ ]:
sequence_ids = inputs.sequence_ids()
sequence_ids


tokenizer 对文本进行了的预处理:
1. Nomalization: 标准化步骤涉及一些常规清理，例如删除不必要的空格、转小写和“/”或删除重音符号。
2. Pre-tokenization: 对原始文本进行初步的分词处理
3. Model: 
4. Postprocessor

## 标准化（normalization）

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print(type(tokenizer.backend_tokenizer))

In [ ]:
print(tokenizer.backend_tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))
